In [ ]:
!pip install kaggle
# Install Kaggle API
!pip install -q kaggle

# Make a directory for the Kaggle API credentials
!mkdir -p ~/.kaggle

# Upload kaggle.json manually
from google.colab import files
files.upload()  # Select the kaggle.json file from your local machine

# Copy the kaggle.json file to the Kaggle API credentials directory
!cp kaggle.json ~/.kaggle/

# Set permissions for the kaggle.json file
!chmod 600 ~/.kaggle/kaggle.json
# Example: Downloading a dataset from Kaggle
# Replace 'dataset-owner/dataset-name' with the specific dataset path
!kaggle competitions download -c haik-24-arabic-sign-language-image-classification

# Unzip the downloaded dataset
!unzip haik-24-arabic-sign-language-image-classification.zip -d /content/haik-24-arabic-sign-language-image-classification
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
# Mount Google Drive
dataset_dir = '/content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification'

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
  inflating: /content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification/toott/toott (562).JPG  
  inflating: /content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification/toott/toott (563).JPG  
  inflating: /content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification/toott/toott (564).JPG  
  inflating: /content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification/toott/toott (565).JPG  
  inflating: /content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification/toott/toott (566).JPG  
  inflating: /content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification/toott/toott (567).JPG  
  inflating: /content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification/t

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os


In [ ]:
# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)



In [ ]:


# Dataset directories
dataset_dir = '/content/haik-24-arabic-sign-language-image-classification/train/train/haik-image claassification'
test_dir = '/content/haik-24-arabic-sign-language-image-classification/test/test'


In [ ]:
# Create ImageDataGenerator objects with more aggressive augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)
# Load and preprocess the data
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 34614 images belonging to 32 classes.


Found 34614 images belonging to 32 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 8638 images belonging to 32 classes.


In [ ]:

# Load the ResNet50 model, excluding the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add new top layers with dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(32, activation='softmax')(x)  # Adjust this to your number of classes

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Use callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')


In [ ]:
# Train the model
history = model.fit(
    train_generator,

    epochs=20,  # Adjust as needed
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)


Epoch 1/20
 433/1082 [===========>..................] - ETA: 17:40 - loss: 2.8452 - accuracy: 0.3010

KeyboardInterrupt: 

In [ ]:
# Unfreeze some of the deeper layers for fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine = model.fit(
    train_generator,
    epochs=20,  # Adjust as needed
    validation_data=validation_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)

NameError: name 'base_model' is not defined

In [ ]:
model.save('final_model.h5')

In [ ]:

# Load the best model
model.load_weights('best_model.h5')

# Plot training & validation accuracy and loss
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

epochs = range(len(acc))

plt.figure()
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
# Save the model for future use
model.save('/content/drive/My Drive/ResNet50_cactus_model.h5')


In [ ]:

# Example of how to use the saved model for predictions
# test_data = '/path/to/test/directory'
# test_generator = train_datagen.flow_from_directory(
#     test_data,
#     target_size=(224, 224),
#     batch_size=32,
#     class_mode='categorical',
#     shuffle=False
# )

# predictions = model.predict(test_generator)
# print(predictions)

In [ ]:
predicitions=best_model.predict(test_generator)

In [ ]:
class_labels={v:k for k,v in train_generator.class_indices.items()}


df_predictions=pd.DataFrame({
    'filename':test_generator.filenames,
    'predicted_class':[class_labels[np.argmax(pred)]for pred in predicitions],
    'confidence' : [np.max(pred) for pred in predicitions]
})

In [ ]:
df_predictions.to_csv('predictions_classification.csv',index=False)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()

# Print final accuracy
print(f"Final validation accuracy: {history_fine.history['val_accuracy'][-1]:.4f}")

# Print classification report
from sklearn.metrics import classification_report
y_true = validation_generator.classes
y_pred = best_model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=class_labels.values()))


In [ ]:
# Monitor and visualize overfitting
def plot_accuracy_difference(history):
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    diff = [train - val for train, val in zip(train_acc, val_acc)]

    plt.figure(figsize=(10, 5))
    plt.plot(diff, label='Training - Validation Accuracy')
    plt.title('Difference between Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy Difference')
    plt.legend()
    plt.show()

    print(f"Maximum overfitting: {max(diff)*100:.2f}%")
    print(f"Final overfitting: {diff[-1]*100:.2f}%")

print("Initial training:")
plot_accuracy_difference(history)
print("\nFine-tuning:")
plot_accuracy_difference(history_fine)



In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print(f"Test accuracy: {test_acc:.4f}")
